In [48]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset

In [49]:
from senmodel.model.utils import convert_dense_to_sparse_network
from senmodel.metrics.edge_finder import EdgeFinder


In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [51]:
from sklearn.preprocessing import LabelEncoder

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
    'hours-per-week', 'native-country', 'income'
]
data = pd.read_csv(url, names=columns, na_values=" ?", skipinitialspace=True)
data = data.dropna()

scaler = StandardScaler()

y = data['occupation']
y = LabelEncoder().fit_transform(y)

X = data.drop(['occupation'], axis=1)

for col in X.select_dtypes(include=['object']).columns:
    X[col] = LabelEncoder().fit_transform(X[col])

X = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

len(set(y))

15

In [52]:
class TabularDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.long)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [53]:
train_dataset = TabularDataset(X_train, y_train)
val_dataset = TabularDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)

In [54]:
class MulticlassFCN(nn.Module):
    def __init__(self, input_size=14, hidden_sizes=None, output_size=15, dropout_rate=0.3):
        super(MulticlassFCN, self).__init__()
        if hidden_sizes is None:
            hidden_sizes = [128, 64]
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.1)

        self.output = nn.Linear(hidden_sizes[1], output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)

        x = self.output(x)
        # x = self.dropout2(x)
        return x


In [55]:
def edge_replacement_func_new_layer(model, optim, val_loader, metric, choose_threshold):
    layer = get_model_last_layer(model)
    ef = EdgeFinder(metric, val_loader, device)
    vals = ef.calculate_edge_metric_for_dataloader(model)
    print("Edge metrics:", vals, max(vals), sum(vals))
    chosen_edges = ef.choose_edges_threshold(model, choose_threshold)
    print("Chosen edges:", chosen_edges, len(chosen_edges[0]))
    layer.replace_many(*chosen_edges)

    if layer.embed_linears:
        optim.add_param_group({'params': layer.embed_linears[-1].weight_values})
    else:
        print("Empty metric")
        dummy_param = torch.zeros_like(layer.weight_values)
        optim.add_param_group({'params': dummy_param})

    return {'max': max(vals), 'sum': sum(vals), 'len': len(vals), 'len_choose': len(chosen_edges[0])}


In [56]:
from senmodel.model.utils import freeze_all_but_last
from tqdm import tqdm
import torch.optim as optim
from sklearn.metrics import accuracy_score


def train_sparse_recursive(model, train_loader, val_loader, num_epochs, metric, edge_replacement_func=None,
                           window_size=3, threshold=0.1, lr=5e-4, choose_threshold=0.3, accumulation_steps=4):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        optimizer.zero_grad()

        for i, (inputs, targets) in enumerate(tqdm(train_loader)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            loss = loss / accumulation_steps
            loss.backward()

            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

            train_loss += loss.item() * accumulation_steps

        if len(train_loader) % accumulation_steps != 0:
            optimizer.step()
            optimizer.zero_grad()

        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0
        all_preds = []
        all_targets = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())

        val_accuracy = accuracy_score(all_targets, all_preds)
        print(f"Epoch {epoch + 1}/{num_epochs} | Train Loss: {train_loss:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.4f}")

        new_l = {}
        val_losses.append(val_loss)
        if edge_replacement_func and len(val_losses) > window_size:
            recent_changes = [abs(val_losses[i] - val_losses[i - 1]) for i in range(-window_size, 0)]
            avg_change = sum(recent_changes) / window_size
            if avg_change < threshold:
                new_l = edge_replacement_func(model, optimizer, val_loader, metric, choose_threshold)
                # Замораживаем все слои кроме последнего
                freeze_all_but_last(model)

        wandb.log({'val_loss': val_loss, 'val_accuracy': val_accuracy, 'train_loss': train_loss} | new_l)

In [57]:
from senmodel.metrics.nonlinearity_metrics import *

criterion = nn.CrossEntropyLoss()
metrics = [
    GradientMeanEdgeMetric(criterion),
    SNIPMetric(criterion),
    MagnitudeL2Metric(criterion),
]

In [58]:
import wandb

wandb.login()

True

In [59]:
hyperparams = {"num_epochs": 50,
               "metric": metrics[0],
               "choose_threshold": 0.05,
               "window_size": 3,
               "threshold": 0.05,
               "lr": 5e-4,
               "accumulation_steps": 1,
               }

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)
name

'num_epochs: 50, metric: GradientMeanEdgeMetric, choose_threshold: 0.05, window_size: 3, threshold: 0.05, lr: 0.0005, accumulation_steps: 2'

In [60]:
dense_model = MulticlassFCN(input_size=X.shape[1])
sparse_model = convert_dense_to_sparse_network(dense_model)
wandb.finish()
wandb.init(
    project="self-expanding-nets",
    name=f"titanic-mul, {name}",
    tags=["complex model", "titanic", "binary", hyperparams["metric"].__class__.__name__],
    group="new freeze"
)

train_sparse_recursive(sparse_model, train_loader, val_loader,
                       edge_replacement_func=edge_replacement_func_new_layer, **hyperparams)
wandb.finish()

100%|██████████| 51/51 [00:01<00:00, 26.24it/s]


Epoch 1/50 | Train Loss: 2.6387 | Val Loss: 32.9973 | Val Accuracy: 0.1882


100%|██████████| 51/51 [00:01<00:00, 30.20it/s]


Epoch 2/50 | Train Loss: 2.4774 | Val Loss: 30.7409 | Val Accuracy: 0.2601


100%|██████████| 51/51 [00:01<00:00, 27.97it/s]


...

100%|██████████| 51/51 [00:34<00:00,  1.46it/s]


Epoch 49/50 | Train Loss: 2.0083 | Val Loss: 24.6889 | Val Accuracy: 0.3493
Edge metrics: tensor([1.0697e-02, 1.6841e-02, 2.2424e-03, 3.4977e-02, 0.0000e+00, 2.6352e-02,
        6.8190e-04, 1.5178e-04, 1.8977e-02, 1.8761e-02, 3.9450e-02, 1.9297e-03,
        1.2966e-02, 4.6280e-02, 2.5786e-02, 2.1873e-02, 2.4679e-02, 3.2803e-02,
        0.0000e+00, 4.3925e-02, 4.0876e-03, 3.5070e-03, 2.8805e-03, 6.4147e-03,
        4.2847e-03, 4.9844e-04, 1.0276e-03, 3.9697e-03, 4.5381e-03, 1.9678e-11,
        4.9253e-03, 2.8215e-03, 1.0213e-10, 2.4938e-11, 3.1019e-12, 1.9979e-12,
        3.4089e-12, 2.4730e-03, 2.6545e-03, 1.4857e-12, 1.4978e-11, 0.0000e+00,
        5.3198e-03, 2.5977e-04, 1.3397e-11, 2.2647e-03, 3.9772e-03, 3.7180e-12,
        9.0689e-04, 5.1009e-03, 2.9483e-11, 9.5354e-11, 5.2509e-03, 1.6893e-03,
        1.9704e-03, 2.7537e-12, 1.4961e-10, 7.7607e-11, 1.0310e-10, 3.1333e-03,
        6.1358e-03, 1.0281e-10, 1.3706e-11, 7.9176e-11, 4.6210e-03, 1.7172e-03,
        1.9325e-03, 7.4866e-12

100%|██████████| 51/51 [00:39<00:00,  1.29it/s]


Epoch 50/50 | Train Loss: 2.0138 | Val Loss: 24.7321 | Val Accuracy: 0.3524
Edge metrics: tensor([7.5304e-03, 1.1347e-02, 1.8666e-03, 2.4106e-02, 0.0000e+00, 1.8964e-02,
        5.0936e-04, 1.0419e-04, 1.3052e-02, 1.2329e-02, 2.7604e-02, 1.4720e-03,
        8.5674e-03, 3.1272e-02, 1.7619e-02, 1.6126e-02, 1.6728e-02, 2.2174e-02,
        0.0000e+00, 3.1651e-02, 3.5235e-03, 3.0231e-03, 2.4830e-03, 5.5295e-03,
        3.6934e-03, 4.2966e-04, 8.8581e-04, 3.4219e-03, 3.9118e-03, 1.3724e-11,
        4.2456e-03, 2.4321e-03, 7.2319e-11, 1.7984e-11, 2.1365e-12, 1.4151e-12,
        2.4364e-12, 2.1317e-03, 2.2882e-03, 1.0561e-12, 1.1047e-11, 0.0000e+00,
        4.5857e-03, 2.2392e-04, 8.9712e-12, 1.9522e-03, 3.4283e-03, 2.5077e-12,
        7.8174e-04, 4.3970e-03, 2.1459e-11, 6.6203e-11, 4.5263e-03, 1.4561e-03,
        1.6985e-03, 1.8865e-12, 1.0607e-10, 5.5113e-11, 7.2249e-11, 2.7009e-03,
        5.2890e-03, 7.1885e-11, 9.1888e-12, 5.6873e-11, 3.9833e-03, 1.4802e-03,
        1.6658e-03, 5.4624e-12

len,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
len_choose,▇█▆▇▆▆▆▆▆▅▅▆▆▂▅▁
max,▄▄▄▂▄▃▃▁▃▄▁▂▂▇▄█
sum,▆█▅▇▅▆▇▅▆▅▆▆█▂▇▁
train_loss,█▆▅▄▃▂▂▂▂▁▁▁▁▁▁▆▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
val_accuracy,▁▄▇▇▇▇█████████▅▆▇▇▇████████████████████
val_loss,█▆▅▃▂▂▂▂▁▁▁▁▁▁▁▁▅▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
len,960
len_choose,549
max,0.84874
sum,93.19083
